# 1. Import Modul

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import joblib


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# import tensorflowjs as tfjs

# 2. Load Data

In [ ]:
data = pd.read_csv("data-preprocessed-berlabel-fix.csv")
data

,tgl-laporan,title,link,content,jenis_laporan,tingkat_bahaya,tingkat_urgensi
0,"Jumat, 08:23",Evakuasi Tawon,https://www.lapor.go.id/laporan/detil/evakuasi...,pernohonan evakuasi tawon atap rumah salah rum...,Evakuasi/Penyelamatan Hewan,Bahaya Rendah,menengah
1,"Jumat, 08:20",ular masuk dapur,https://www.lapor.go.id/laporan/detil/ular-mas...,laporan salah warga kelurahan mondokan kecamat...,Evakuasi/Penyelamatan Hewan,Bahaya Rendah,menengah
2,"Jumat, 08:18",ular masuk rumah,https://www.lapor.go.id/laporan/detil/ular-mas...,laporan salah warga kelurahan ronggomulyo keca...,Evakuasi/Penyelamatan Hewan,Bahaya Rendah,menengah
3,"Jumat, 08:15",Kunci tertinggal di dalam mobil,https://www.lapor.go.id/laporan/detil/kunci-te...,laporan salah warga desa tunah kecamatan seman...,Penyelamatan Non Hewan & Bantuan Teknis,Bahaya Rendah,rendah
4,"Jumat, 08:12",Evakuasi Ular Sawo Kembang,https://www.lapor.go.id/laporan/detil/evakuasi...,laporan permohonan evakuasi ular sowo kembang ...,Evakuasi/Penyelamatan Hewan,Bahaya Rendah,menengah
...,...,...,...,...,...,...,...
711,1/8/2025 0:00,"Bantuan Untuk kelurahan pejagalan, Penjaringan...",https://www.lapor.go.id/laporan/detil/bantuan-...,"tolong bantu bantuan kelurahan pejagalan , kec...",Layanan Lingkungan & Fasilitas Umum,Bahaya Rendah,rendah
712,1/8/2025 0:00,Permohonan Bantuan Penanganan Banjir,https://www.lapor.go.id/laporan/detil/permohon...,selamat pagi dinas penanggulangan bencana daer...,Layanan Lingkungan & Fasilitas Umum,Bahaya Rendah,menengah
713,1/8/2025 0:00,Banjir Cideng Akibat Luapan Dari 2 Kali,https://www.lapor.go.id/laporan/detil/banjir-c...,"selamat pagi . , air banjir cideng lambat suru...",Layanan Lingkungan & Fasilitas Umum,Bahaya Rendah,menengah
714,1/8/2025 0:00,Permohonan Bantuan Penanganan Banjir,https://www.lapor.go.id/laporan/detil/permohon...,"shalom , air banjir cideng lambat surut . kelu...",Layanan Lingkungan & Fasilitas Umum,Bahaya Rendah,menengah


In [31]:
data.dropna(inplace=True)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(data['content'], data['jenis_laporan'], test_size=0.2, random_state=42)

# 3. Load Pretrained Model

In [34]:
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
model = TFBertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=len(data['jenis_laporan'].unique()))

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 4. Data Encoding & Konfigurasi Model

In [35]:
# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Prepare data for BERT
def encode_data(tokenizer, texts, max_len=128):
    return tokenizer(
        list(texts),
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=True,
        return_tensors='tf'
    )

X_train_encoded = encode_data(tokenizer, X_train)
X_test_encoded = encode_data(tokenizer, X_test)

In [36]:
joblib.dump(label_encoder, '/content/drive/MyDrive/Coding_Camp/Capstone_Project/label_encoder.pkl')

['/content/drive/MyDrive/Coding_Camp/Capstone_Project/label_encoder.pkl']

In [37]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=2e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# 5. Train Model

In [39]:
# Train the model
history = model.fit(
    [X_train_encoded.input_ids, X_train_encoded.token_type_ids, X_train_encoded.attention_mask],
    y_train_encoded,
    validation_data=(
        [X_test_encoded.input_ids, X_test_encoded.token_type_ids, X_test_encoded.attention_mask],
        y_test_encoded
    ),
    epochs=15,
    batch_size=32,
)

Epoch 1/15
18/18 [==============================] - 61s 1s/step - loss: 1.0447 - accuracy: 0.5632 - val_loss: 0.6085 - val_accuracy: 0.9021
Epoch 2/15
18/18 [==============================] - 15s 824ms/step - loss: 0.3927 - accuracy: 0.9105 - val_loss: 0.1932 - val_accuracy: 0.9301
Epoch 3/15
18/18 [==============================] - 15s 828ms/step - loss: 0.1499 - accuracy: 0.9561 - val_loss: 0.1103 - val_accuracy: 0.9720
Epoch 4/15
18/18 [==============================] - 15s 835ms/step - loss: 0.0794 - accuracy: 0.9807 - val_loss: 0.1043 - val_accuracy: 0.9790
Epoch 5/15
18/18 [==============================] - 15s 850ms/step - loss: 0.0369 - accuracy: 0.9947 - val_loss: 0.1166 - val_accuracy: 0.9790
Epoch 6/15
18/18 [==============================] - 15s 860ms/step - loss: 0.0241 - accuracy: 0.9947 - val_loss: 0.1385 - val_accuracy: 0.9650
Epoch 7/15
18/18 [==============================] - 15s 859ms/step - loss: 0.0115 - accuracy: 1.0000 - val_loss: 0.1442 - val_accuracy: 0.9650
Ep

# 6. Evaluasi

In [46]:
loss, accuracy = model.evaluate(
    [X_test_encoded.input_ids, X_test_encoded.token_type_ids, X_test_encoded.attention_mask],
    y_test_encoded
)

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Get predictions
y_pred_logits = model.predict(
    [X_test_encoded.input_ids, X_test_encoded.token_type_ids, X_test_encoded.attention_mask]
).logits
y_pred_encoded = np.argmax(y_pred_logits, axis=1)

# Decode predictions and true labels
y_pred = label_encoder.inverse_transform(y_pred_encoded)
y_true = label_encoder.inverse_transform(y_test_encoded)

# Print classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(y_true, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

5/5 [==============================] - 1s 222ms/step - loss: 0.1811 - accuracy: 0.9650
Test Loss: 0.1811467409133911
Test Accuracy: 0.9650349617004395
5/5 [==============================] - 4s 221ms/step

Classification Report:
                                         precision    recall  f1-score   support

            Evakuasi/Penyelamatan Hewan       1.00      1.00      1.00        71
                              Kebakaran       1.00      0.97      0.99        37
    Layanan Lingkungan & Fasilitas Umum       0.82      1.00      0.90        23
Penyelamatan Non Hewan & Bantuan Teknis       1.00      0.67      0.80        12

                               accuracy                           0.97       143
                              macro avg       0.96      0.91      0.92       143
                           weighted avg       0.97      0.97      0.96       143


Confusion Matrix:
[[71  0  0  0]
 [ 0 36  1  0]
 [ 0  0 23  0]
 [ 0  0  4  8]]


In [47]:
# Example inference function
def predict_label(text):
    encoded_text = encode_data(tokenizer, [text])
    pred_logits = model.predict(
        [encoded_text.input_ids, encoded_text.token_type_ids, encoded_text.attention_mask]
    ).logits
    pred_encoded = np.argmax(pred_logits, axis=1)
    predicted_label = label_encoder.inverse_transform(pred_encoded)
    return predicted_label[0]

## Example usage:
sample_text = "ada kalajengking pak dalam rumah saya. lokasi jalan jeruk no.9"
predicted_category = predict_label(sample_text)
print(f"The predicted category for the text '{sample_text}' is: {predicted_category}")

sample_text_2 = "tolong pak kebakaran di jalan sudirman"
predicted_category_2 = predict_label(sample_text_2)
print(f"The predicted category for the text '{sample_text_2}' is: {predicted_category_2}")

sample_text_3 = "ada orang terjebak dalam lift yang rusak pak. lokasi gedung honda"
predicted_category_3 = predict_label(sample_text_3)
print(f"The predicted category for the text '{sample_text_3}' is: {predicted_category_3}")

sample_text_4 = "tolong pak. ada orang mau bunuh diri lompat dari monas"
predicted_category_4 = predict_label(sample_text_4)
print(f"The predicted category for the text '{sample_text_4}' is: {predicted_category_4}")

sample_text_5 = "ada ular kobra pak dirumah saya. alamat jalan mangga no.4"
predicted_category_5 = predict_label(sample_text_5)
print(f"The predicted category for the text '{sample_text_5}' is: {predicted_category_5}")

1/1 [==============================] - 0s 56ms/step
The predicted category for the text 'ada kalajengking pak dalam rumah saya. lokasi jalan jeruk no.9' is: Penyelamatan Non Hewan & Bantuan Teknis
1/1 [==============================] - 0s 53ms/step
The predicted category for the text 'tolong pak kebakaran di jalan sudirman' is: Kebakaran
1/1 [==============================] - 0s 55ms/step
The predicted category for the text 'ada orang terjebak dalam lift yang rusak pak. lokasi gedung honda' is: Penyelamatan Non Hewan & Bantuan Teknis
1/1 [==============================] - 0s 54ms/step
The predicted category for the text 'tolong pak. ada orang mau bunuh diri lompat dari monas' is: Penyelamatan Non Hewan & Bantuan Teknis
1/1 [==============================] - 0s 54ms/step
The predicted category for the text 'ada ular kobra pak dirumah saya. alamat jalan mangga no.4' is: Evakuasi/Penyelamatan Hewan


# 7. Save Model

In [50]:
# Dummy input untuk membangun model dengan sequence length 128
dummy_input = {
    "input_ids": tf.constant([[1]*128], dtype=tf.int32),
    "token_type_ids": tf.constant([[0]*128], dtype=tf.int32),
    "attention_mask": tf.constant([[1]*128], dtype=tf.int32)
}

# Panggil model untuk mengunci input shape
_ = model(dummy_input)

# Simpan ulang model dengan input shape tetap
model.save("/content/drive/MyDrive/Coding_Camp/Capstone_Project/models/experiment3/saved_model")

In [51]:
loaded = tf.saved_model.load("/content/drive/MyDrive/Coding_Camp/Capstone_Project/models/experiment3/saved_model")
print(loaded.signatures["serving_default"].structured_input_signature)

((), {'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int32, name='attention_mask'), 'input_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name='input_ids'), 'token_type_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name='token_type_ids')})


In [52]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.allow_custom_ops = True
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter.experimental_fixed_shape = True  # Optional
tflite_model = converter.convert()

with open('/content/drive/MyDrive/Coding_Camp/Capstone_Project/models/experiment3/model.tflite', 'wb') as f:
    f.write(tflite_model)


# 8. Inferensi dengan Save Model dan TFLite

In [57]:
# Load the saved model
loaded_model = tf.saved_model.load('/content/drive/MyDrive/Coding_Camp/Capstone_Project/models/experiment3/saved_model')
infer = loaded_model.signatures["serving_default"]

# Load the label encoder
label_encoder = joblib.load('/content/drive/MyDrive/Coding_Camp/Capstone_Project/label_encoder.pkl')

# Define the inference function using the loaded model
def predict_label_loaded_model(text):
    encoded_text = encode_data(tokenizer, [text])
    # The input tensor names might vary depending on how the model was saved.
    # You might need to inspect the loaded_model.signatures["serving_default"].structured_input_signature
    # to get the exact input tensor names.
    # Assuming the input names are 'input_ids', 'token_type_ids', and 'attention_mask'
    predictions = infer(
        input_ids=encoded_text.input_ids,
        token_type_ids=encoded_text.token_type_ids,
        attention_mask=encoded_text.attention_mask
    )

    # The output tensor name might also vary.
    # Assuming the output tensor name is 'logits'
    pred_logits = predictions['logits'].numpy()
    pred_encoded = np.argmax(pred_logits, axis=1)
    predicted_label = label_encoder.inverse_transform(pred_encoded)
    return predicted_label[0]

## Example usage:
sample_text = "ada kalajengking pak dalam rumah saya. lokasi jalan jeruk no.9"
predicted_category = predict_label_loaded_model(sample_text)
print(f"The predicted category for the text '{sample_text}' is: {predicted_category}")

sample_text_2 = "tolong pak kebakaran di jalan sudirman"
predicted_category_2 = predict_label_loaded_model(sample_text_2)
print(f"The predicted category for the text '{sample_text_2}' is: {predicted_category_2}")

sample_text_3 = "ada orang terjebak dalam lift yang rusak pak. lokasi gedung honda"
predicted_category_3 = predict_label_loaded_model(sample_text_3)
print(f"The predicted category for the text '{sample_text_3}' is: {predicted_category_3}")

sample_text_4 = "pohon tumbang depan kejaksaan pak"
predicted_category_4 = predict_label_loaded_model(sample_text_4)
print(f"The predicted category for the text '{sample_text_4}' is: {predicted_category_4}")

sample_text_5 = "ada ular kobra pak dirumah saya. alamat jalan mangga no.4"
predicted_category_5 = predict_label_loaded_model(sample_text_5)
print(f"The predicted category for the text '{sample_text_5}' is: {predicted_category_5}")

The predicted category for the text 'ada kalajengking pak dalam rumah saya. lokasi jalan jeruk no.9' is: Penyelamatan Non Hewan & Bantuan Teknis
The predicted category for the text 'tolong pak kebakaran di jalan sudirman' is: Kebakaran
The predicted category for the text 'ada orang terjebak dalam lift yang rusak pak. lokasi gedung honda' is: Penyelamatan Non Hewan & Bantuan Teknis
The predicted category for the text 'pohon tumbang depan kejaksaan pak' is: Layanan Lingkungan & Fasilitas Umum
The predicted category for the text 'ada ular kobra pak dirumah saya. alamat jalan mangga no.4' is: Evakuasi/Penyelamatan Hewan


In [65]:
# Load TFLite model
interpreter = tf.lite.Interpreter(model_path="/content/drive/MyDrive/Coding_Camp/Capstone_Project/models/experiment3/model.tflite")
interpreter.allocate_tensors()

# Get input/output info
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Function for prediction using resize method (the working one)
def predict_label_tflite(text, max_len=128):
    """
    Predict label using TFLite model with tensor resizing
    """
    # Resize input tensors to accept longer sequences
    interpreter.resize_tensor_input(0, [1, max_len])  # attention_mask
    interpreter.resize_tensor_input(1, [1, max_len])  # input_ids
    interpreter.resize_tensor_input(2, [1, max_len])  # token_type_ids
    interpreter.allocate_tensors()

    # Tokenize text
    encoded = tokenizer(
        [text],
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='np'
    )

    # Convert to int32 for TFLite
    input_ids = encoded['input_ids'].astype(np.int32)
    token_type_ids = encoded['token_type_ids'].astype(np.int32)
    attention_mask = encoded['attention_mask'].astype(np.int32)

    # Set tensors (based on the working order from your output)
    interpreter.set_tensor(input_details[0]['index'], attention_mask)  # attention_mask first
    interpreter.set_tensor(input_details[1]['index'], input_ids)       # input_ids second
    interpreter.set_tensor(input_details[2]['index'], token_type_ids)  # token_type_ids third

    # Run inference
    interpreter.invoke()

    # Get output and predict
    output = interpreter.get_tensor(output_details[0]['index'])
    pred_encoded = np.argmax(output, axis=1)
    predicted_label = label_encoder.inverse_transform(pred_encoded)

    return predicted_label[0]

# Test dengan contoh teks
sample_texts = [
    "permisi pak, ada kebakaran di kantor kejaksaan",
    "ular masuk rumah pak",
    "kebakaran di menara kembar Pak",
    "ada kucing terjebak di atap rumah",
    "pohon tumbang menghalangi jalan raya",
    "teman saya terjebak dalam lift pak, meminta bantuan evakuasi penyelamatan"
]

print("=== Hasil Prediksi TFLite Model ===")
for i, text in enumerate(sample_texts, 1):
    try:
        prediction = predict_label_tflite(text)
        print(f"{i}. Text: '{text}'")
        print(f"   Predicted Category: {prediction}")
        print()
    except Exception as e:
        print(f"{i}. Text: '{text}'")
        print(f"   Error: {e}")
        print()

=== Hasil Prediksi TFLite Model ===
1. Text: 'permisi pak, ada kebakaran di kantor kejaksaan'
   Predicted Category: Kebakaran

2. Text: 'ular masuk rumah pak'
   Predicted Category: Evakuasi/Penyelamatan Hewan

3. Text: 'kebakaran di menara kembar Pak'
   Predicted Category: Kebakaran

4. Text: 'ada kucing terjebak di atap rumah'
   Predicted Category: Evakuasi/Penyelamatan Hewan

5. Text: 'pohon tumbang menghalangi jalan raya'
   Predicted Category: Layanan Lingkungan & Fasilitas Umum

6. Text: 'teman saya terjebak dalam lift pak, meminta bantuan evakuasi penyelamatan'
   Predicted Category: Penyelamatan Non Hewan & Bantuan Teknis

